In [3]:
from langchain_groq import ChatGroq

In [5]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='your_real_api_key_here',
    model_name = "llama-3.3-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, as part of the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [6]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-33460")
page_data = loader.load().pop().page_content
print(page_data)





















Nike Careers








































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



Select a Languag

In [7]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [8]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'ATHLETE ADDETTO ALLE VENDITE 21H - CATEGORIA PROTETTA (L.68/99) - NFS',
  'experience': 'Part-time',
  'skills': 'Sales, Customer Service',
  'description': 'Work at Nike Store in Mantova Outlet Village'},
 {'role': 'ATHLETE PART TIME 20H - FIDENZA',
  'experience': 'Part-time',
  'skills': 'Sales, Customer Service',
  'description': 'Work at Nike Store in Fidenza'},
 {'role': 'Account Executive Jordan, Pacific',
  'experience': 'Full-time',
  'skills': 'Sales, Customer Service, Account Management',
  'description': 'Work as Account Executive for Jordan brand in Melbourne, Australia'},
 {'role': 'Accounting Admin - Nike Jersey Gardens',
  'experience': 'Full-time',
  'skills': 'Accounting, Administration',
  'description': 'Work as Accounting Admin at Nike Jersey Gardens in Elizabeth, New Jersey'},
 {'role': 'Administrative Assistant',
  'experience': 'Full-time',
  'skills': 'Administration, Microsoft Office',
  'description': 'Work as Administrative Assistant in Karnataka,

In [9]:
type(json_res)

list

In [10]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [17]:
import uuid
import os
import chromadb

# Get path to user's home directory
home_dir = os.path.expanduser("~")
persist_dir = os.path.join(home_dir, "vectorstore")
os.makedirs(persist_dir, exist_ok=True)

# Use the correct and accessible path
client = chromadb.PersistentClient(path=persist_dir)

# Assuming df is defined somewhere
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(
            documents=[row["Techstack"]],
            metadatas={"links": row["Links"]},
            ids=[str(uuid.uuid4())]
        )


In [25]:
if isinstance(json_res, list):
    job = json_res[0]
else:
    job = json_res

links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])


In [26]:
job

{'role': 'ATHLETE ADDETTO ALLE VENDITE 21H - CATEGORIA PROTETTA (L.68/99) - NFS',
 'experience': 'Part-time',
 'skills': 'Sales, Customer Service',
 'description': 'Work at Nike Store in Mantova Outlet Village'}

TypeError: list indices must be integers or slices, not str

In [30]:
# Prepare the cold email prompt
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}

    ### INSTRUCTION:
    You are Mohan, a Business Development Executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
    the seamless integration of business processes through automated tools.
    Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability,
    process optimization, cost reduction, and heightened overall efficiency.
    Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ
    in fulfilling their needs.
    Also add the most relevant ones from the following links to showcase AtliQ's portfolio: {link_list}
    Remember you are Mohan, BDE at AtliQ.
    Do not provide a preamble.
    ### EMAIL (NO PREAMBLE):
    """
)

# Chain the prompt with the model
chain_email = prompt_email | llm

# Invoke the chain with the job description and portfolio links
res = chain_email.invoke({
    "job_description": str(job),
    "link_list": links
})

# Display the generated cold email
print(res.content)

Subject: Enhance Your Business Operations with AtliQ's Automated Solutions

Dear Hiring Manager,

I came across the job postings for various roles at Nike, including ATHLETE ADDETTO ALLE VENDITE, ATHLETE PART TIME, Account Executive, Accounting Admin, Administrative Assistant, and Analyst positions. As a Business Development Executive at AtliQ, I believe our company can help streamline your business processes and enhance overall efficiency.

At AtliQ, we specialize in providing AI-powered and software consulting services to facilitate seamless integration of business operations. Our team of experts has empowered numerous enterprises with tailored solutions, resulting in scalability, process optimization, cost reduction, and heightened overall efficiency. We can help Nike optimize its sales, customer service, accounting, and administrative processes, among others.

Our portfolio showcases our capabilities in delivering high-quality solutions. For instance, our Magento portfolio (https:/